In [2]:
import mlflow
from mlflow import MlflowClient

EXPERIMENT = 'student_performance'
TRACKING_SERVER_HOST = "localhost"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")
mlflow.set_experiment(EXPERIMENT)
MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

In [3]:
client = MlflowClient(MLFLOW_TRACKING_URI)

model_name = "student-performance-predictor"
model_alias = 'best'

client.get_model_version_by_alias(model_name, model_alias)

<ModelVersion: aliases=['best'], creation_timestamp=1728357935108, current_stage='None', description='', last_updated_timestamp=1728357935108, name='student-performance-predictor', run_id='69b183aad246467c8decdb5419d3c36c', run_link='', source='mlflow-artifacts:/1/69b183aad246467c8decdb5419d3c36c/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>

In [6]:
model_version_info = client.get_model_version_by_alias(model_name, model_alias)
model_version = model_version_info.version
RUN_ID = model_version_info.run_id


model_uri = f"models:/{model_name}/{model_version}"
model = mlflow.pyfunc.load_model(model_uri)

2024/10/08 03:47:31 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - numpy (current: 2.1.2, required: numpy==1.26.4)
 - pandas (current: 2.2.3, required: pandas==2.1.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [7]:
RUN_ID

'69b183aad246467c8decdb5419d3c36c'